In [1]:
import sqlite3
import pandas as pd

# Create in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create employees table
cursor.execute('''
CREATE TABLE employees (
    emp_id INTEGER PRIMARY KEY AUTOINCREMENT,
    emp_name TEXT NOT NULL,
    department TEXT NOT NULL,
    salary REAL NOT NULL,
    hire_date TEXT NOT NULL
)
''')

# Insert sample employee data
employees_data = [
    ('Alice Johnson', 'Sales', 90000, '2022-01-15'),
    ('Bob Smith', 'Engineering', 120000, '2021-06-01'),
    ('Carol White', 'Sales', 85000, '2022-03-10'),
    ('David Brown', 'Engineering', 130000, '2020-11-20'),
    ('Eve Davis', 'Marketing', 75000, '2023-02-14'),
    ('Frank Wilson', 'Sales', 95000, '2021-09-05')
]

cursor.executemany(
    'INSERT INTO employees (emp_name, department, salary, hire_date) VALUES (?, ?, ?, ?)',
    employees_data
)
conn.commit()

print("✓ Employee database created successfully")
print(f"✓ {len(employees_data)} employees inserted")

✓ Employee database created successfully
✓ 6 employees inserted


## Subqueries (Nested Queries)

A query within another query. Can be in SELECT, FROM, or WHERE clause.

**WHERE Subquery**: Filter results based on subquery results
```sql
SELECT * FROM employees 
WHERE salary > (SELECT AVG(salary) FROM employees);
```

In [2]:
# Find employees earning above average salary
result = pd.read_sql_query(
    '''SELECT emp_name, department, salary 
       FROM employees 
       WHERE salary > (SELECT AVG(salary) FROM employees)
       ORDER BY salary DESC''',
    conn
)
print("Employees earning above average:")
print(result)
print()

# Find employees in departments with highest average salary
result = pd.read_sql_query(
    '''SELECT emp_name, department, salary
       FROM employees
       WHERE department IN (SELECT department FROM employees GROUP BY department ORDER BY AVG(salary) DESC LIMIT 1)
       ORDER BY salary DESC''',
    conn
)
print("Employees in highest-paying department:")
print(result)

Employees earning above average:
      emp_name   department    salary
0  David Brown  Engineering  130000.0
1    Bob Smith  Engineering  120000.0

Employees in highest-paying department:
      emp_name   department    salary
0  David Brown  Engineering  130000.0
1    Bob Smith  Engineering  120000.0


## FROM Subqueries (Derived Tables)

Use subquery results as a temporary table.

```sql
SELECT * FROM (
  SELECT department, AVG(salary) as avg_sal FROM employees GROUP BY department
) as dept_stats
WHERE avg_sal > 100000;
```

In [3]:
# Department salary analysis
result = pd.read_sql_query(
    '''SELECT department, avg_salary, min_salary, max_salary, emp_count
       FROM (
         SELECT department, 
                ROUND(AVG(salary), 2) as avg_salary,
                MIN(salary) as min_salary,
                MAX(salary) as max_salary,
                COUNT(*) as emp_count
         FROM employees 
         GROUP BY department
       ) as dept_stats
       ORDER BY avg_salary DESC''',
    conn
)
print("Department Salary Statistics:")
print(result)

Department Salary Statistics:
    department  avg_salary  min_salary  max_salary  emp_count
0  Engineering    125000.0    120000.0    130000.0          2
1        Sales     90000.0     85000.0     95000.0          3
2    Marketing     75000.0     75000.0     75000.0          1


## Common Table Expressions (CTEs) with WITH Clause

Named temporary result sets that improve readability and reusability.

```sql
WITH dept_stats AS (
  SELECT department, AVG(salary) as avg_sal FROM employees GROUP BY department
)
SELECT e.emp_name, e.salary, d.avg_sal
FROM employees e
JOIN dept_stats d ON e.department = d.department;
```

In [4]:
# CTE to compare employee salary with department average
result = pd.read_sql_query(
    '''WITH dept_stats AS (
         SELECT department, ROUND(AVG(salary), 2) as avg_salary
         FROM employees
         GROUP BY department
       )
       SELECT e.emp_name, e.department, e.salary, d.avg_salary,
              ROUND(e.salary - d.avg_salary, 2) as difference
       FROM employees e
       JOIN dept_stats d ON e.department = d.department
       ORDER BY e.department, difference DESC''',
    conn
)
print("Employee Salary vs Department Average:")
print(result)

Employee Salary vs Department Average:
        emp_name   department    salary  avg_salary  difference
0    David Brown  Engineering  130000.0    125000.0      5000.0
1      Bob Smith  Engineering  120000.0    125000.0     -5000.0
2      Eve Davis    Marketing   75000.0     75000.0         0.0
3   Frank Wilson        Sales   95000.0     90000.0      5000.0
4  Alice Johnson        Sales   90000.0     90000.0         0.0
5    Carol White        Sales   85000.0     90000.0     -5000.0


## Views (Virtual Tables)

Named queries that can be queried like regular tables.

```sql
CREATE VIEW high_earners AS
SELECT emp_name, department, salary
FROM employees
WHERE salary > 100000;

SELECT * FROM high_earners;
```

In [5]:
# Create views for different purposes
cursor.execute('''
CREATE VIEW high_earners AS
SELECT emp_name, department, salary
FROM employees
WHERE salary > 100000
''')

cursor.execute('''
CREATE VIEW dept_salaries AS
SELECT department,
       ROUND(AVG(salary), 2) as avg_salary,
       ROUND(MIN(salary), 2) as min_salary,
       ROUND(MAX(salary), 2) as max_salary,
       COUNT(*) as emp_count
FROM employees
GROUP BY department
''')
conn.commit()

print("Views created: high_earners, dept_salaries")
print()

# Query the high_earners view
result = pd.read_sql_query('SELECT * FROM high_earners ORDER BY salary DESC', conn)
print("High Earners (salary > $100,000):")
print(result)
print()

# Query the dept_salaries view
result = pd.read_sql_query('SELECT * FROM dept_salaries ORDER BY avg_salary DESC', conn)
print("Department Salary Summary:")
print(result)

Views created: high_earners, dept_salaries

High Earners (salary > $100,000):
      emp_name   department    salary
0  David Brown  Engineering  130000.0
1    Bob Smith  Engineering  120000.0

Department Salary Summary:
    department  avg_salary  min_salary  max_salary  emp_count
0  Engineering    125000.0    120000.0    130000.0          2
1        Sales     90000.0     85000.0     95000.0          3
2    Marketing     75000.0     75000.0     75000.0          1


## Transactions and ACID Properties

Ensure data consistency through BEGIN, COMMIT, and ROLLBACK.

```sql
BEGIN TRANSACTION;
UPDATE employees SET salary = salary * 1.1 WHERE department = 'Sales';
UPDATE employees SET salary = salary * 0.95 WHERE department = 'Marketing';
COMMIT;  -- or ROLLBACK to undo
```

In [6]:
# Example 1: Successful transaction
print("Original Salaries:")
result = pd.read_sql_query('SELECT emp_name, department, salary FROM employees WHERE department = "Sales"', conn)
print(result)
print()

try:
    cursor.execute('BEGIN')
    cursor.execute('UPDATE employees SET salary = salary * 1.1 WHERE department = "Sales"')
    cursor.execute('COMMIT')
    print("✓ Transaction committed: Sales salaries increased by 10%")
except Exception as e:
    cursor.execute('ROLLBACK')
    print(f"✗ Transaction rolled back: {e}")
print()

print("Updated Salaries:")
result = pd.read_sql_query('SELECT emp_name, department, salary FROM employees WHERE department = "Sales"', conn)
print(result)

Original Salaries:
        emp_name department   salary
0  Alice Johnson      Sales  90000.0
1    Carol White      Sales  85000.0
2   Frank Wilson      Sales  95000.0

✓ Transaction committed: Sales salaries increased by 10%

Updated Salaries:
        emp_name department    salary
0  Alice Johnson      Sales   99000.0
1    Carol White      Sales   93500.0
2   Frank Wilson      Sales  104500.0


## Indexes for Performance

Create indexes on frequently searched columns to speed up queries.

```sql
CREATE INDEX idx_dept ON employees(department);
CREATE INDEX idx_salary ON employees(salary);
CREATE INDEX idx_name ON employees(emp_name);
```

In [7]:
# Create indexes
cursor.execute('CREATE INDEX idx_department ON employees(department)')
cursor.execute('CREATE INDEX idx_salary ON employees(salary)')
cursor.execute('CREATE INDEX idx_name ON employees(emp_name)')
conn.commit()

print("✓ Indexes created:")
print("  - idx_department on employees(department)")
print("  - idx_salary on employees(salary)")
print("  - idx_name on employees(emp_name)")
print()

# List all indexes
result = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='index' AND tbl_name='employees'",
    conn
)
print("Existing Indexes on employees table:")
print(result)

✓ Indexes created:
  - idx_department on employees(department)
  - idx_salary on employees(salary)
  - idx_name on employees(emp_name)

Existing Indexes on employees table:
             name
0  idx_department
1      idx_salary
2        idx_name


## Query Execution Analysis

Use EXPLAIN QUERY PLAN to understand how queries are executed.

```sql
EXPLAIN QUERY PLAN SELECT * FROM employees WHERE salary > 100000;
```

In [8]:
# Analyze query execution plan
queries = [
    ('Index scan on salary', 'SELECT * FROM employees WHERE salary > 100000'),
    ('Index scan on department', 'SELECT * FROM employees WHERE department = "Engineering"'),
    ('Full table scan', 'SELECT * FROM employees WHERE UPPER(emp_name) = "ALICE JOHNSON"')
]

for description, query in queries:
    print(f"Query: {description}")
    result = pd.read_sql_query(f'EXPLAIN QUERY PLAN {query}', conn)
    print(result.to_string(index=False))
    print()

Query: Index scan on salary
 id  parent  notused                                             detail
  3       0      203 SEARCH employees USING INDEX idx_salary (salary>?)

Query: Index scan on department
 id  parent  notused                                                     detail
  3       0       62 SEARCH employees USING INDEX idx_department (department=?)

Query: Full table scan
 id  parent  notused         detail
  2       0      216 SCAN employees



## Advanced SQL Concepts Summary

| Concept | Purpose | Use Case |
|---------|---------|----------|
| **Subqueries** | Nested SELECT statements | Filter by aggregate, complex WHERE conditions |
| **CTEs (WITH)** | Named temporary result sets | Improve readability, reusable result sets |
| **Views** | Virtual tables | Hide complexity, provide security layer |
| **Transactions** | Group operations | Ensure data consistency, ACID compliance |
| **Indexes** | Speed up searches | Improve SELECT performance on large tables |
| **EXPLAIN PLAN** | Analyze execution | Optimize slow queries, understand strategy |

## Performance Tips
1. Use indexes on frequently filtered columns
2. CTEs are more readable than nested subqueries
3. Views help abstract complexity and improve maintainability
4. Transactions ensure ACID properties in multi-step operations
5. Use EXPLAIN QUERY PLAN to identify slow queries

In [9]:
# View complete employee data
result = pd.read_sql_query('SELECT * FROM employees ORDER BY salary DESC', conn)
print("Complete Employee Data:")
print(result)

Complete Employee Data:
   emp_id       emp_name   department    salary   hire_date
0       4    David Brown  Engineering  130000.0  2020-11-20
1       2      Bob Smith  Engineering  120000.0  2021-06-01
2       6   Frank Wilson        Sales  104500.0  2021-09-05
3       1  Alice Johnson        Sales   99000.0  2022-01-15
4       3    Carol White        Sales   93500.0  2022-03-10
5       5      Eve Davis    Marketing   75000.0  2023-02-14
